In [13]:
import torch

d_model = 12
trg_len = 1
BATCH_SIZE = 5 

In [14]:
V = torch.randn(BATCH_SIZE, trg_len, d_model, requires_grad=True)  # For simplicity, a 3-dimensional input
V

tensor([[[-1.0870,  1.4679, -0.3964,  0.3793,  0.5043,  0.9177,  0.7692,
          -0.1930, -0.7334,  0.8734, -0.3822,  0.1088]],

        [[-0.9930,  0.6887, -1.5292, -0.3639, -0.1337, -0.9362, -1.3257,
          -0.0454,  0.3081, -0.7320,  0.1541, -0.5236]],

        [[-0.9973,  0.3035, -1.2115, -1.8725, -1.1464,  0.3051,  0.7094,
           0.4515,  0.5623, -0.6957, -1.0634,  0.4820]],

        [[-0.4527, -1.5459, -0.2140,  0.7529, -0.7922, -0.2014, -0.3007,
          -0.7866,  0.7521,  0.3239, -0.0395,  0.5355]],

        [[ 0.0168,  0.5925, -1.3263,  1.0311, -0.7999, -0.1513,  0.3619,
          -0.5641, -0.1595, -0.5207,  0.1579, -0.3582]]], requires_grad=True)

In [15]:
def g(V):
    return torch.sin(V)  # Example non-linear function

Y = g(V)

In [16]:
print(V.shape)
print(V.reshape(-1).shape)

torch.Size([5, 1, 12])
torch.Size([60])


[source link](https://gist.github.com/apaszke/226abdf867c4e9d6698bd198f3b45fb7)

In [17]:
def jacobian(y, x, create_graph=False):                                                               
    jac = []                                                                                          
    flat_y = y.reshape(-1)                                                                            
    grad_y = torch.zeros_like(flat_y)                                                                 
    for i in range(len(flat_y)):                                                                      
        grad_y[i] = 1.                                                                                
        grad_x, = torch.autograd.grad(flat_y, x, grad_y, retain_graph=True, create_graph=create_graph)
        jac.append(grad_x.reshape(x.shape))                                                           
        grad_y[i] = 0.                                                                                
    return torch.stack(jac).reshape(y.shape + x.shape)                                                
                                                                                                      
def hessian(y, x):                                                                                    
    return jacobian(jacobian(y, x, create_graph=True), x)                                             
                                                                                                      
def f(x):                                                                                             
    return x * x * torch.arange(4, dtype=torch.float)                                                 
                                                                                                      
x = torch.ones(4, requires_grad=True)
                                                                 
print(jacobian(f(x), x))                                                                              
print(hessian(f(x), x))      

tensor([[0., 0., 0., 0.],
        [0., 2., 0., 0.],
        [0., 0., 4., 0.],
        [0., 0., 0., 6.]])
tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 2., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 4., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 6.]]])


In [12]:
# Initialize a list to collect gradients for each component of Y
jacobian = []

for y in Y:
    # Compute gradient of each y component with respect to V
    grad_y = torch.autograd.grad(y, V, retain_graph=True, create_graph=True)[0]
    jacobian.append(grad_y)

# Stack the results to form the Jacobian matrix
jacobian = torch.stack(jacobian)  # Shape: [output_dim, input_dim]
jacobian

RuntimeError: grad can be implicitly created only for scalar outputs